**Integrantes: Erich Gonzalez, Daniel Gutierrez San José y Georgelys Marcano

Enunciado: 

    - Calcular mediante paralelizacion con Spark la media y varianza del dataset data_ok.csv utilizando exclusivamente funciones basicas map/reduce (textFile, reduce, reduceByKey, map, flatmap, filter, count).

    - Calcular inicialmente para una sola columna y mas tarde para todas las columnas del dataset.

    - Verificar que la solucion propuesta es correcta con las funciones rdd.mean() y rdd.stdev().

Se deben realizar 3 versiones:

    a1: Calcula la media y varianza para la columna 2

    a2: utilizando las operaciones de vectorizacion de python y arrays de numpy, utilizar la misma estructura de codigo de la version 1 para calcular las medias y varianzas de todas las columnas

    a3: Transforma cada celda del dataset en un elemento (j,v), donde "j" es la columna de la celda y "v" es el valor de la celda del rdd. Resuelve el problema con esta nueva estructura del dataset

### Versión a3: Transforma cada celda del dataset en un elemento (j,v), donde "j" es la columna de la celda y "v" es el valor de la celda del rdd. Resuelve el problema con esta nueva estructura del dataset

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
import numpy as np

In [2]:
sc = SparkContext("local[*]", "version3a")

24/09/25 20:51:11 WARN Utils: Your hostname, DESKTOP-KLPDI5O resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/25 20:51:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 20:51:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
path =  "./data_ok.csv"
file = sc.textFile(path)

In [12]:
rdd = file.map(lambda linea:linea.split())
# transformar a float
datasetfloat = rdd.map(lambda row: [float(celda) for celda in row])
tuplas = datasetfloat.map(lambda row:[(columna,valor) for columna,valor in enumerate(row)])
#tuplas.take(2)
suma = tuplas.reduce(lambda row1,row2:[(indice,item[1]+row2[indice][1]) for indice,item in enumerate(row1)])
#print(suma)
total_item_rows = datasetfloat.count()
media_col = []
for index,tupla in enumerate(suma):
    media = tupla[1] / total_item_rows
    media_col.append(media)
#print("Media de cada columna:", media_col)
# Calcular la varianza
suma_diferencias_cuadradas = tuplas.map(lambda row: [(indice, (valor[1] - media_col[indice])**2) for indice, valor in enumerate(row)])
# Reducir para sumar las diferencias cuadradas
suma_varianza = suma_diferencias_cuadradas.reduce(lambda row1, row2: [(indice, item[1] + row2[indice][1]) for indice, item in enumerate(row1)])
# Calcular la varianza dividiendo entre el total de filas
varianza_col = [tupla[1] / total_item_rows for tupla in suma_varianza]
print(varianza_col)

[401818.16628544027, 3168373338.3683057, 2959605.6432505627, 5883755024032.205, 0.0, 2.3010223599999864, 0.0, 313332.59728123934, 19828.36648764016, 345513253204.2377, 6514263.902446111, 0.03228335999999504, 0.05090556000000112, 3.9563996399999914, 14631.804300159674, 19686496.994590227, 103958.43979824062, 12358118332394.264, 435068936.0129385, 0.0, 114088.27606048658, 151781717098.3398, 3110.345729247053, 19567004.17901126, 0.0, 105249.50629016865, 11237.074611699474, 19550727.745639034, 0.0, 215925.38426967297, 14448.852004224205, 19659327.47487623, 596.8763070399867, 202491.0921141331, 176553635.66167575, 590364421.4027784, 43583.33676315991, 40518057.64093468, 3760978.8000936317, 19624991.613284435, 1923635.5490620334, 1689189.9964354113, 401750.96461244015, 3174031454.5406246, 2959811.9491644404, 5884337014379.24, 0.0, 2.3098521600000868, 0.0, 310335.4895564397, 20014.45445755969, 345325149316.8787, 6514263.902446111, 0.03228335999999504, 0.04515323999999891, 3.599737439999948, 1